In [25]:
import xarray as xr
import glob

def preprocess_dataset(ds: xr.Dataset) -> xr.Dataset:
    """
    Preprocess the dataset to ensure the `valid_time` coordinate is in `datetime64[ns]` format.

    Args:
        ds (xr.Dataset): Input dataset.

    Returns:
        xr.Dataset: Preprocessed dataset with `valid_time` converted to datetime.
    """
    if "valid_time" in ds.coords:
        ds["valid_time"] = ds["valid_time"].dt.floor("D")  # Truncate to date only
    return ds

# Define the directory containing your NetCDF files
data_dir = 'soil-water-forecasting/modeling-pipeline/data/01_raw/unzipped_files/ERA5_monthly_averaged_data_on_single_levels/'

# Create a sorted list of file paths
file_list = sorted(glob.glob(data_dir + '*.nc'))

# Open multiple files as a single dataset with preprocessing applied
ds = xr.open_mfdataset(
    file_list, 
    combine='by_coords', 
    preprocess=preprocess_dataset
)


In [26]:
ds

<xarray.Dataset> Size: 53GB
Dimensions:     (valid_time: 550, latitude: 721, longitude: 1440)
Coordinates:
    number      int64 8B 0
  * valid_time  (valid_time) datetime64[ns] 4kB 1979-01-01 ... 2024-10-01
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver      (valid_time) <U4 9kB dask.array<chunksize=(550,), meta=np.ndarray>
Data variables: (12/23)
    cl          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    cvh         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    tvh         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    asn         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    rsn         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    sst         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    ...          ...
    ssr         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    e           (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    ro          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    pev         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    swvl1       (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

# Format datatype

In [28]:
# Update swvl1: Set to 1 wherever lsm indicates water (lsm == 1)
ds['swvl1'] = ds['swvl1'].where(ds['lsm'] != 0, other=1)

# Fill na values

In [29]:
# INTO 3.data_preprocessing.ipynb 

import xarray as xr
from typing import List, Union

def fillna_in_variables(
    ds: xr.Dataset,
    variables: List[str],
    fill_value: Union[int, float]
) -> xr.Dataset:
    """
    Fills NaN values in specified variables of an xarray.Dataset with a provided value.

    Args:
        ds (xr.Dataset): The input dataset containing the data variables.
        variables (List[str]): A list of variable names in the dataset for which to fill NaN values.
        fill_value (Union[int, float]): The value to fill NaN values with (e.g., 0).

    Returns:
        xr.Dataset: A new dataset with NaN values filled in the specified variables.
    
    Raises:
        ValueError: If a variable in `variables` does not exist in the dataset.
    
    Example:
        filled_ds = fillna_in_variables(ds, ["t2m", "swvl1"], fill_value=0)
    """
    # Check if each variable exists in the dataset
    for var in variables:
        if var not in ds:
            raise ValueError(f"Variable '{var}' not found in the dataset.")
    
    # Fill NaN values for the specified variables
    filled_ds = ds.copy()
    for var in variables:
        filled_ds[var] = filled_ds[var].fillna(fill_value)
    
    return filled_ds

In [30]:
ds = fillna_in_variables(ds, ["sst"], fill_value=0)

# Drop irrelavat variables 

In [31]:
ds = ds.drop_vars(['number', 'expver'])

In [32]:
ds

<xarray.Dataset> Size: 53GB
Dimensions:     (valid_time: 550, latitude: 721, longitude: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 4kB 1979-01-01 ... 2024-10-01
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
Data variables: (12/23)
    cl          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    cvh         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    tvh         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    asn         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    rsn         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    sst         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    ...          ...
    ssr         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    e           (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    ro          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    pev         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    swvl1       (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

# Interploation

In [33]:
import numpy as np
import xarray as xr

# Step 1: Define target grid with 1° resolution
target_lat = np.arange(-90, 90.1, 1)   # From -90 to 90 degrees inclusive
target_lon = np.arange(0, 360, 1)      # From 0 to 359.5 degrees inclusive

# Step 2: Create target grid Dataset (optional, for reference)
target_grid = xr.Dataset(
    {
        "latitude": (["latitude"], target_lat),
        "longitude": (["longitude"], target_lon),
    }
)

# Step 3: Ensure latitude is ascending
if ds.latitude[0] > ds.latitude[-1]:
    ds = ds.sortby("latitude")

# Step 4: Perform interpolation
ds = ds.interp(latitude=target_lat, longitude=target_lon, method="linear")


In [34]:
ds

<xarray.Dataset> Size: 3GB
Dimensions:     (valid_time: 550, latitude: 181, longitude: 360)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 4kB 1979-01-01 ... 2024-10-01
  * latitude    (latitude) float64 1kB -90.0 -89.0 -88.0 ... 88.0 89.0 90.0
  * longitude   (longitude) int64 3kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
Data variables: (12/23)
    cl          (valid_time, latitude, longitude) float32 143MB dask.array<chunksize=(92, 181, 360), meta=np.ndarray>
    cvh         (valid_time, latitude, longitude) float32 143MB dask.array<chunksize=(92, 181, 360), meta=np.ndarray>
    tvh         (valid_time, latitude, longitude) float32 143MB dask.array<chunksize=(92, 181, 360), meta=np.ndarray>
    asn         (valid_time, latitude, longitude) float32 143MB dask.array<chunksize=(92, 181, 360), meta=np.ndarray>
    rsn         (valid_time, latitude, longitude) float32 143MB dask.array<chunksize=(92, 181, 360), meta=np.ndarray>
    sst         (valid_time, latitude, longitude) float32 143MB dask.array<chunksize=(92, 181, 360), meta=np.ndarray>
    ...          ...
    ssr         (valid_time, latitude, longitude) float32 143MB dask.array<chunksize=(92, 181, 360), meta=np.ndarray>
    e           (valid_time, latitude, longitude) float32 143MB dask.array<chunksize=(92, 181, 360), meta=np.ndarray>
    ro          (valid_time, latitude, longitude) float32 143MB dask.array<chunksize=(92, 181, 360), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 143MB dask.array<chunksize=(92, 181, 360), meta=np.ndarray>
    pev         (valid_time, latitude, longitude) float32 143MB dask.array<chunksize=(92, 181, 360), meta=np.ndarray>
    swvl1       (valid_time, latitude, longitude) float32 143MB dask.array<chunksize=(92, 181, 360), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

# Smaller data range for testing 

In [35]:
# Filter the dataset for the date range 2016-01-01 to 2023-12-01
ds = ds.sel(valid_time=slice("2016-01-01", "2022-12-01"))

In [36]:
ds

<xarray.Dataset> Size: 504MB
Dimensions:     (valid_time: 84, latitude: 181, longitude: 360)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 672B 2016-01-01 ... 2022-12-01
  * latitude    (latitude) float64 1kB -90.0 -89.0 -88.0 ... 88.0 89.0 90.0
  * longitude   (longitude) int64 3kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
Data variables: (12/23)
    cl          (valid_time, latitude, longitude) float32 22MB dask.array<chunksize=(16, 181, 360), meta=np.ndarray>
    cvh         (valid_time, latitude, longitude) float32 22MB dask.array<chunksize=(16, 181, 360), meta=np.ndarray>
    tvh         (valid_time, latitude, longitude) float32 22MB dask.array<chunksize=(16, 181, 360), meta=np.ndarray>
    asn         (valid_time, latitude, longitude) float32 22MB dask.array<chunksize=(16, 181, 360), meta=np.ndarray>
    rsn         (valid_time, latitude, longitude) float32 22MB dask.array<chunksize=(16, 181, 360), meta=np.ndarray>
    sst         (valid_time, latitude, longitude) float32 22MB dask.array<chunksize=(16, 181, 360), meta=np.ndarray>
    ...          ...
    ssr         (valid_time, latitude, longitude) float32 22MB dask.array<chunksize=(16, 181, 360), meta=np.ndarray>
    e           (valid_time, latitude, longitude) float32 22MB dask.array<chunksize=(16, 181, 360), meta=np.ndarray>
    ro          (valid_time, latitude, longitude) float32 22MB dask.array<chunksize=(16, 181, 360), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 22MB dask.array<chunksize=(16, 181, 360), meta=np.ndarray>
    pev         (valid_time, latitude, longitude) float32 22MB dask.array<chunksize=(16, 181, 360), meta=np.ndarray>
    swvl1       (valid_time, latitude, longitude) float32 22MB dask.array<chunksize=(16, 181, 360), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

In [38]:
ds.to_netcdf('soil-water-forecasting/modeling-pipeline/data/02_intermediate/preprocessed_data.nc')